# Session 2 (Fall 2024)

## Outline

- Review

  * Geometric distribution: Coupon Collection
  * Hash functions
  * Universal Hash Families

- Similarity and Minhashing

  * Jaccard similarity and distance
  * Minhashing
    
- Locality Sensitive Hashing

  * Banding signatures
  * S-curves


In [1]:
import numpy as np
import pandas as pd

rng = np.random.default_rng()

## Review: Geometric Distribution

> Start with a coin where the probability of Heads is $p$

Geometric RV: Number of independent coin tosses before Heads appears for the first time.

- Expected Value? Variance?

- How to sample from a geometric distribution?

- **Class Problem 1**

- Example Application: The Coupon Collector problem!

- Validating the algorithm for counting distinct items in a stream



## Coupon Collector Problem

Another gem from probability theory with lots of applications! There are $n$ different types of coupons and each time you visit the store, you get one coupon (of some type) with uniform probability. How many visits to the store are required before you get *each* different type of coupon for the first time? 

**Simulation:** Mimic the coupon collection problem for $n=50$ and repeat the experiment (of collecting all $n$ coupons) 1000 times and plot the mean number of store visits. See anything interesting? You can analyze this exactly using an interesting summation called the **Harmonic sum**:

$$H_n = 1 + \frac{1}{2} + \ldots + \frac{1}{n}$$

which is almost exactly (upto an additive constant called Euler's constant) equal to $\ln n$. Hence, the number of store visits on average, is $n (\ln n + 0.577)$.


## Simulating Coupon Collection for $n=50$

- The first visit always yields an unseen coupon.

- To collect the next new coupon, we simulate a geometric rv with $p=49/50$.

- Continue this 49 times, with $p$ decreasing by $1/50$ each time.

**Trick:** Instead of tossing coins to mimic a geometric rv, we can get an appropriate value with just one randomly generated number between 0 and 1!

In [2]:
## Simulate a geometric rv
q = 0.75
rand_val = rng.random()
print(rand_val)
x = int(np.ceil(np.log(rand_val)/np.log(q)))
x

0.23073437054636436


6

In [3]:
## Simulate 1000 trials of coupon collection for 50 coupons

experiment = rng.random((1000, 50))

In [4]:
experiment[:2,:3]

array([[0.13705405, 0.9523082 , 0.97093041],
       [0.73995789, 0.69931634, 0.59918745]])

### Hashing

- Hash functions: keys are hashed to buckets 
- Open-addressing (one key per bucket)
- Hashing with chaining (colliding keys chained in bucket)
- Assumptions: uniform hashing
- Used very widely to construct efficient data structures for storing big data (indexes).

#### Collision

> Occurs when keys $x \not= y$ end up hashing to the same slot! $$h(x) = h(y)$$

- Collisions are inevitable, especially when we have no prior knowledge of $V$. 

- Distribution of the number of keys that collide in different slots can vary wildly!

> **Ideal distribution:** collisions distributed *uniformly*.

In [5]:
# Examples of hash functions

def h(key, m):
    """Returns slot index for key
    
    Args:
        key (int): number to be hashed
        m (int): table size    
    """
    return key % m

def g(d, s):
    """Returns 32-bit number as slot index
    
    Uses the FNV algorithm from http://isthe.com/chongo/tech/comp/fnv/ 
    
    Args:
        d (int)
        s (str): key to be hashed
    
    """
    if d == 0: 
        d = 0x01000193
    for c in s:
        d = ( (d * 0x01000193) ^ ord(c) ) & 0xffffffff
    return d

In [6]:
g(0, 'Jaden')

1542760701

### Universal hash family

We start by defining a universal family of **hash functions**. This a family with the property that:

> for any pair of keys in the universe of keys (i.e. $U$), a hash function drawn uniformly at random from the family will cause the keys to collide with an expected probability of $1/m$, where $m$ is the table size.

Let $p$ be a large prime, and let $a, b$ be random integers chosen uniformly such that $1 \leq a \leq p-1$ and $0 \leq b \leq p-1$.

In [7]:

class UHF:
    """A factory for producing a universal family of hash functions"""

    @staticmethod
    def is_prime(k):
        if k%2==0:
            return False
        for i in range(3, int(np.sqrt(k)), 2):
            if k%i == 0:
                return False
        return True
    
    def __init__(self, n):
        """Universe size is n"""
        self.n = n
        self.rng = np.random.default_rng()
        if n%2==0:
            m = n+1
        else:
            m = n+2
        while not(UHF.is_prime(m)):
            m = m+2
        self.p = m
        
    def make_hash(self, m):
        """Return a random hash function
        
        m: table size
        """
        a = rng.integers(1,self.p)
        b = rng.integers(0,self.p)
        return lambda k: ((a*k+b)%self.p)%m

In [8]:
factory = UHF(1000000)

In [9]:
factory.p

1000003

### Test the Universal Hash Family (Exercise)

Set up and evaluate an experiment to verify that the family defined above is indeed a universal family. Use a reasonable universe of keys, say integers in $range(1000000)$ and a hash table of size $5000$. 

- Fix two keys at random
- Generate a lot of hash functions and see how many of them collide 

In [10]:
key1 = rng.integers(0, 1000000)
print(key1)
key2 = rng.integers(0, 1000000)
print(key2)
num_collisions = 0
factory = UHF(1000000)
for _ in range(10000):
    hash_fn = factory.make_hash(5000)
    if hash_fn(key1) == hash_fn(key2):
        num_collisions += 1
print(num_collisions)

675777
153443
3


### Universal hash families

We start by defining a universal family of **hash functions**. This is a family with the following property:

> Fix a pair of distinct keys in $U$. Then, a function *drawn uniformly at random* from the family will cause the keys to collide with an **expected probability** of $1/m$.

## Similarity and Minhashing

### Data Mining

Given lots of data, discover patterns and models that are:

- **Valid:**  hold on new data with some certainty
- **Useful:**  should be possible to act on the item 
- **Unexpected:**  non-obvious to the system
- **Understandable:** humans should be able to interpret the pattern

#### Data as Feature Matrices

- Each data item is generically referred to as a **document**

- In the simplest case, documents are *characterized* by the presence or absence of *binary* features (indicated by 0 and 1), i.e., the data is the **characteristic matrix** of the features.


In [11]:
characters = pd.DataFrame({'Hermione': [1,0,0,1,1,0,1],
                           'Harry': [0,0,1,1,1,0,1],
                           'Ron': [1,1,0,0,0,1,1],
                           'Severus': [1,0,0,0,0,1,1]})
                           
                           

In [12]:
characters

,Hermione,Harry,Ron,Severus
0,1,0,1,1
1,0,0,1,0
2,0,1,0,0
3,1,1,0,0
4,1,1,0,0
5,0,0,1,1
6,1,1,1,1


### Jaccard Similarity

Jaccard *similarity* between two documents $D_i$ and $D_j$ equals 
$$ \mbox{SIM}(i,j) = \frac{|D_i \cap D_j|}{|D_i \cup D_j|}.$$



In [13]:
f_matrix = characters.to_numpy()

**Question:** What is the Jaccard similarity between `Harry` and `Hermione`? Between `Severus` and `Ron`?

#### Compressing a document's representation: Minhash Signatures

Choose a *random* permutation $p$ of the features. A **minhash signature** 
$${\mbox{sign}}_p(i)$$
for document $D_i$ using permutation $p$ is obtained as follows:

- recall that the document is a feature vector in the original order of features
- **Reorder** the feature vector according to the permutation $p$
- Among all features present in the document, determine the one with the **smallest index in the new ordering**! This index is the signature for the document.

#### Example

In [14]:
f_matrix = characters.to_numpy()
f_matrix

array([[1, 0, 1, 1],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       [1, 1, 0, 0],
       [1, 1, 0, 0],
       [0, 0, 1, 1],
       [1, 1, 1, 1]])

For our example above, consider document `Hermione` with feature vector 
$$[1,0,0,1,1,0,1]$$ (the first column above).

Consider the *permutation* of the **feature indices** given by 
$$[3,1,0,6,2,5,4].$$

This says that feature 3 will be numbered 0, feature 1 still numbered 1, feature 0 numbered 2 and so on. Hence, the permutation will **reorder** document `Hermione` as
$$[\underline{1},0,1,1,0,0,1]$$
e.g. the underlined 1 at the new index **0**, corresponds to the original feature number 3. 

If we apply the permutation across all characters (columns), we get:

In [15]:
perm = [3,1,0,6,2,5,4]
perm_mat = np.take(f_matrix, perm, axis=0)
perm_mat

array([[1, 1, 0, 0],
       [0, 0, 1, 0],
       [1, 0, 1, 1],
       [1, 1, 1, 1],
       [0, 1, 0, 0],
       [0, 0, 1, 1],
       [1, 1, 0, 0]])

In [16]:
f_matrix

array([[1, 0, 1, 1],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       [1, 1, 0, 0],
       [1, 1, 0, 0],
       [0, 0, 1, 1],
       [1, 1, 1, 1]])

To determine the **signature** of document `Hermione` according to permutation `perm` above, we just walk down the *permuted* column until we find the **first row index** corresponding to a feature that `Hermione` *contains*, viz. the 1 entry at index **0**. 

Doing this for all columns, the signatures for the documents (in order) according to `perm` are: 0, 0, 1, 2.

In [17]:
np.argmin(1 - perm_mat, axis=0)

array([0, 0, 1, 2])

## Questions

- What are the signatures for the other documents according to the permutation $[6,4,2,0,5,3,1]?
- What signatures are possible, in theory, for `Harry`?
- What **common signatures** are possible for `Harry` and `Severus`?


In [18]:
f_matrix

array([[1, 0, 1, 1],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       [1, 1, 0, 0],
       [1, 1, 0, 0],
       [0, 0, 1, 1],
       [1, 1, 1, 1]])

### Using numpy to compute document signatures

We can do this in a variety of ways. For example:

1. Start with a random permutation $p$ of the row indices. Apply the permutation $p$ to the matrix, then determine the signature by using `argmin` over the matrix with entries *flipped* (to find the first 1).

In [19]:
p_mat = rng.permutation(f_matrix, axis=0)
np.argmin(1 - p_mat, axis=0)

array([0, 0, 1, 1])

2. We need not actually permute the rows of the matrix at all - permuting rows can be very expensive with large data. Instead, we imagine that the rows are numbered from 1 through $m$, the number of features. A permutation can be thought of as supplying **weights** for the corresponding features. We multiply each feature by its weight, then find the smallest non-zero weighted entry in a column: that is the signature! 

In [20]:
## Mask all 0 values
n_rows = f_matrix.shape[0]
masked = np.where(f_matrix == 0, n_rows+1, f_matrix)
masked

array([[1, 8, 1, 1],
       [8, 8, 1, 8],
       [8, 1, 8, 8],
       [1, 1, 8, 8],
       [1, 1, 8, 8],
       [8, 8, 1, 1],
       [1, 1, 1, 1]])

In [21]:
f_matrix

array([[1, 0, 1, 1],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       [1, 1, 0, 0],
       [1, 1, 0, 0],
       [0, 0, 1, 1],
       [1, 1, 1, 1]])

In [22]:
perm_arr = np.array(perm) + 1  # make elements non-zero
permuted_tr = masked.T * perm_arr  # transpose to do elementwise multiplication

In [23]:
permuted_tr

array([[ 4, 16,  8,  7,  3, 48,  5],
       [32, 16,  1,  7,  3, 48,  5],
       [ 4,  2,  8, 56, 24,  6,  5],
       [ 4, 16,  8, 56, 24,  6,  5]])

In [24]:
signs = np.min(permuted_tr.T, axis=0)

In [25]:
signs

array([3, 1, 2, 4])

## Discussion (5 minutes)

We used the same permutation `perm` = [3,1,0,6,2,5,4] but seem to get different signatures. 
Why is that? Discuss in your group why this is happening. Understand the code in the cells below to frame the discussion.


In [26]:
perm_inv = np.argsort(perm)
perm_inv

array([2, 1, 4, 0, 6, 5, 3])

In [27]:
np.argmin(1 - np.take(f_matrix, perm, axis=0), axis=0)

array([0, 0, 1, 2])

### Relationship between Jaccard similarity and signatures

Given a *random* permutation $p$, the Jaccard similarity, $\mbox{SIM}(i,j)$ is related to signatures by the following property:

> $$\mbox{Pr}\{{\mbox{sign}}_p(i) = {\mbox{sign}}_p(j)\} = \mbox{SIM}(i,j)$$

Thus, the likelihood of two documents being deemed similar (via their signatures) will be in accordance with their (mathematical) similarity!

**Proof:** In class!

## Minhashing

Based on the proof, we can see that if we were to calculate a large number of **different signatures** (each corresponding to a different permutation), we can ***approximate the true Jaccard similarity of two documents as the fraction of permutations with matching signatures.*** 

Random permutations are very expensive to compute: it takes time linear in $n$ to obtain such a permutation. When $n$ is very large, this is infeasible.

So, we **approximate the signature** by using a **random hash function** in
place of a random permutation!

> Use the **range** of a hash function applied to the feature indices as a **proxy** for permuting the indices of the features. 

#### Example

Let's use our universal hash family!

In [35]:
uhf = UHF(1000)
uhf.p

1009

In [40]:
hfn = np.vectorize(uhf.make_hash(53))

In [41]:
hfn(np.arange(7))

array([14, 19, 22, 27, 32, 35, 40])

## Minhash Signatures

Minhashing using just one small signature will unfortunately produce both **false positives** and **false negatives** for similarity. We try to mitigate this as follows:

- generate **many** small signatures using different hash functions

*Minhash similarity* between two documents is the expected proportion of hash
functions for which their small signatures agree!

#### Jaccard Distance

The Jaccard distance between two documents $i$ and $j$ is given by $$d(i,j)=1 -
\mbox{SIM}(i,j)$$. 

It is distance *metric*:

- it equals 0 iff $D_i = D_j$
- it is symmetric: $d(i,j)=d(j,i)$
- it satisfies the *triangle inequality*: $d(i,j) ~<=~ d(i,k) ~+~ d(k,j)$

#### Exercise

- Compute signature matrices for a document using both random permutations and hash functions

- Use the matrices to see how accurately they approximate the true jaccard similarity.


In [53]:
def sign(mat, h_fn, p):
    """
    Return array of signatures for all documents

    The shape of the matrix is (m, n) where m = number of documents
    and n = number of features. The hash function has domain equal to
    the column indices 0, 1, ..., n-1
    
    mat (np.array): 0/1 matrix
    h_fn (int -> int): universal hash function whose range is 0, ..., p-1
    p (int): upper bound on values returned by hfn 
    """
    n_features = mat.shape[1]
    masked = np.where(mat == 0, n_features + 1, mat)
    print(masked)
    s = masked*np.vectorize(h_fn)(np.arange(n_features))
    return np.min(s, 1)
    

In [54]:
tbl_size = 53
uhf = UHF(tbl_size)
h = uhf.make_hash(tbl_size)

In [55]:
f_matrix.T

array([[1, 0, 0, 1, 1, 0, 1],
       [0, 0, 1, 1, 1, 0, 1],
       [1, 1, 0, 0, 0, 1, 1],
       [1, 0, 0, 0, 0, 1, 1]])

In [56]:
signatures = sign(f_matrix.T, h, tbl_size)

[[1 8 8 1 1 8 1]
 [8 8 1 1 1 8 1]
 [1 1 8 8 8 1 1]
 [1 8 8 8 8 1 1]]


In [57]:
signatures

array([ 2,  2, 16, 16])